# Imports and program variables


In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics
import pandas as pd

from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers import Dense
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint

from time_series_dataset_loader import TimeSeriesDatasetLoader

from sklearn.model_selection import train_test_split # to split dataset into train and test
from sklearn import preprocessing
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.
/home/moutinho/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/moutinho/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/moutinho/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/moutinho/.local/lib/python3.6/site-packages/tensorflo

In [3]:
DATASET_PATH = '../datasets/MFCC/'

## Setting up train and test datasets

In [4]:
dataset_loader = TimeSeriesDatasetLoader(DATASET_PATH)
X, y = dataset_loader.get_dataset()

In [5]:
X = np.asarray(X)
y = np.asarray(y)

In [6]:
# Completing with zeros
max_len = len(X[0])
for row in X:
        if len(row) > max_len:
            max_len = len(row)

X = pad_sequences(X, maxlen=max_len, padding='post')

In [7]:
# See X and y details
print(X[0][0])
print(X.shape)

print(y[:3])
print(y.shape)

[  0   0  -7  19  -7   1   5 -13   3 -10  13  -1   7   0   2  -2]
(1066, 543, 16)
[3 0 1]
(1066,)


In [8]:
# Split the dataset in train and test
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3)

print("\nX_train:\n")
print(X_train[0][0])
print(X_train.shape)

print("\nX_test:\n")
print(X_test[0][0])
print(X_test.shape)

print("\ny_train:\n")
print(y_train[:3])
print(y_train.shape)

print("\ny_test:\n")
print(y_test[:3])
print(y_test.shape)


X_train:

[  0   0  -9   2   7   1  14   7  25   0  10   6   3   0   0 -15]
(746, 543, 16)

X_test:

[  0   0   2  12  11   4   8   8  -1   4   9 -16   2   6  -7 -27]
(320, 543, 16)

y_train:

[4 5 1]
(746,)

y_test:

[3 6 2]
(320,)


In [9]:
# Create categorical matrices
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [10]:
num_rows = 543
num_columns = 16
num_channels = 1

# X_train = X_train.reshape(X_train.shape[0], num_rows, num_columns, num_channels)
# X_test = X_test.reshape(X_test.shape[0], num_rows, num_columns, num_channels)

print("\nX_train:\n")
print(X_train[0][0])
print(X_train.shape)

print("\nX_test:\n")
print(X_test[0][0])
print(X_test.shape)


X_train:

[  0   0  -9   2   7   1  14   7  25   0  10   6   3   0   0 -15]
(746, 543, 16)

X_test:

[  0   0   2  12  11   4   8   8  -1   4   9 -16   2   6  -7 -27]
(320, 543, 16)


## Building and Compiling Model

In [11]:
# Construct model 1
model = Sequential()

model.add(LSTM(units=128, dropout=0.05, recurrent_dropout=0.35, return_sequences=True, input_shape=(num_rows, num_columns)))
model.add(LSTM(units=32,  dropout=0.05, recurrent_dropout=0.35, return_sequences=False))
model.add(Dense(units=7, activation="softmax"))

model.add(Dense(7, activation='softmax'))

In [12]:
# # top-k category accuracy
# top3_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=3)
# top3_acc.__name__ = 'top3_acc'

# compile the keras model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
# Define bath and epochs
batch_size = 128
epochs = 100

lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=20, min_lr=0.0000001)
mcp_save = ModelCheckpoint('../models/model_checkpoints/time_series.h5', save_best_only=True, monitor='val_loss', mode='min')
cnnhistory=model.fit(X_train, y_train, batch_size = batch_size, epochs = epochs, validation_data=(X_test, y_test), callbacks=[mcp_save, lr_reduce])


Train on 746 samples, validate on 320 samples
Epoch 1/100
746/746 [==============================] - 14s 19ms/step - loss: 1.9638 - accuracy: 0.1314 - val_loss: 1.9461 - val_accuracy: 0.1688
Epoch 2/100
746/746 [==============================] - 13s 17ms/step - loss: 1.9596 - accuracy: 0.1327 - val_loss: 1.9459 - val_accuracy: 0.1688
Epoch 3/100
746/746 [==============================] - 13s 17ms/step - loss: 1.9509 - accuracy: 0.1394 - val_loss: 1.9548 - val_accuracy: 0.1063
Epoch 4/100
746/746 [==============================] - 13s 18ms/step - loss: 1.9477 - accuracy: 0.1555 - val_loss: 1.9593 - val_accuracy: 0.1063
Epoch 5/100
746/746 [==============================] - 14s 18ms/step - loss: 1.9486 - accuracy: 0.1542 - val_loss: 1.9575 - val_accuracy: 0.1063
Epoch 6/100
746/746 [==============================] - 13s 18ms/step - loss: 1.9460 - accuracy: 0.1367 - val_loss: 1.9570 - val_accuracy: 0.1500
Epoch 7/100
746/746 [==============================] - 13s 18ms/step - loss: 1.9445

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

# Model Summary

model.summary()

# PLT History info
plt.plot(cnnhistory.history['loss'])
plt.plot(cnnhistory.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()